# Customer Outreach Campaign

We learn how to create custom tools for a customer outreach campaign. A Good tool has 3 key elements:

- Versatility
- Caching
- Fault Tolerance

Note: CrewAI only works with Python version >=3.10 and <=3.13 

In [ ]:
import sys
print(sys.version)

## Step 01. Installing CrewAI packages

In [ ]:
%%capture
!pip install crewai

In [ ]:
%%capture
!pip install 'crewai[tools]'

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Step 02. Importing CrewAI library (Agent, Task and Crew module)

In [ ]:
from crewai import Agent, Task, Crew

To use Agents, we need to set env. variables (key, model name)

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'Your OPENAI API KEY'
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
os.environ['SERPER_API_KEY'] = 'Your SERPER API KEY'

## Step 03. Creating Agents

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Step 04. Creating Tools

### CrewAI Tools

- **SerperDevTool** : Used for Google Search
- **DirectoryReadTool** : Open a directory
- **FileReadTool** : Read a file

In [ ]:
from crewai_tools import SerperDevTool, DirectoryReadTool, FileReadTool

In [ ]:
# local directory 'instructions' contains markdown files
# markdown files (.md) having instructions on how to deal with company of different sizes 

directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
serper_dev_tool = SerperDevTool()

### Custom Tool (SentimentAnalysisTool)

- Create a custom tool using crewAI's BaseTool class

In [ ]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- Tool logic written in `_run` function
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name : str = "Sentiment Analysis Tool"
    description : str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, argument : str) -> str:
        return "positive"

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Step 05. Creating Tasks

- Lead Profiling Task usign crewAI Tools

In [ ]:
lead_profiling = Task (
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[serper_dev_tool, directory_read_tool, file_read_tool],
    agent=sales_rep_agent
)

- The personalized outreach task is created using custom tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search tool)

In [ ]:
personalized_outreach_task = Task (
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[serper_dev_tool, sentiment_analysis_tool],
    agent=lead_sales_rep_agent
)

## Step 06.  Creating a crew 

In [ ]:
crew = Crew(
    tasks=[lead_profiling, personalized_outreach_task],
    agents=[sales_rep_agent, lead_sales_rep_agent],
    memory=True,
    verbose=2
)

## Kicking off the crew

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)